In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
BASE_DIR = '../Datasets'
GLOVE_DIR = BASE_DIR + '/glove.6B/'
TEXT_DATA_DIR = BASE_DIR + '/mpqa535/'
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [3]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [4]:
import csv
texts = []
y = []
for root, dirs, files in os.walk('../Datasets/mpqa535/'):
    for f in files:
        with open(os.path.join(root,f)) as csvfile:
            reader = csv.reader(csvfile)
            reader.next()
            for row in reader:
                texts.append(row[0])
                y.append(0 if row[1] == 'f' else 1)

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [6]:
X = pad_sequences(sequences)
y = np.asarray(y)

In [7]:
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]
num_validation_samples = int(VALIDATION_SPLIT * X.shape[0])
x_train = X[:-num_validation_samples]
y_train = y[:-num_validation_samples]
x_val = X[-num_validation_samples:]
y_val = y[-num_validation_samples:]

In [8]:
num_words = len(word_index)
embedding_matrix = np.zeros((num_words+1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [9]:
embedding_layer = Embedding(num_words+1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=X.shape[1],
                            trainable=False)

In [10]:
from keras import backend as K

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)

In [11]:
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding

sequence_input = Input(shape=(X.shape[1],), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 3, activation='relu')(embedded_sequences)
x = MaxPooling1D(2)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(2)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(31)(x)

x = Flatten()(x)

preds = Dense(1, activation='sigmoid')(x)

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc',precision, recall, fmeasure])

model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 8886 samples, validate on 2221 samples
Epoch 1/10
 256/8886 [..............................] - ETA: 2030s - loss: 0.7556 - acc: 0.4414 - precision: 0.3624 - recall: 0.4593 - fmeasure: 0.3539       

/home/projjal/anaconda2/lib/python2.7/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.394409). Check your callbacks.
  % delta_t_median)


8886/8886 [==============================] - 68s - loss: 0.5425 - acc: 0.7219 - precision: 0.7418 - recall: 0.7774 - fmeasure: 0.7424 - val_loss: 0.4996 - val_acc: 0.7425 - val_precision: 0.8154 - val_recall: 0.6802 - val_fmeasure: 0.7362
Epoch 2/10
8886/8886 [==============================] - 8s - loss: 0.4535 - acc: 0.7842 - precision: 0.7997 - recall: 0.8049 - fmeasure: 0.7948 - val_loss: 0.4849 - val_acc: 0.7668 - val_precision: 0.7445 - val_recall: 0.8674 - val_fmeasure: 0.7976
Epoch 3/10
8886/8886 [==============================] - 8s - loss: 0.4093 - acc: 0.8129 - precision: 0.8287 - recall: 0.8346 - fmeasure: 0.8221 - val_loss: 0.4770 - val_acc: 0.7596 - val_precision: 0.7956 - val_recall: 0.7484 - val_fmeasure: 0.7668
Epoch 4/10
8886/8886 [==============================] - 8s - loss: 0.3357 - acc: 0.8490 - precision: 0.8652 - recall: 0.8635 - fmeasure: 0.8579 - val_loss: 0.4943 - val_acc: 0.7650 - val_precision: 0.7761 - val_recall: 0.7979 - val_fmeasure: 0.7820
Epoch 5/10
888